# count min sketch source :
https://github.com/rafacarrascosa/countminsketch

In [106]:
import pip
pip.main(['install', "countminsketch"])

0

In [1]:
from countminsketch import CountMinSketch
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import math

w =  number of hash functions used
p = number of elements in hash table

In [4]:
np.random.seed(10)
p= 10000
w = 4

In [5]:
waveform_data = pd.read_csv("sanitized_waveform.data",header = None)

Count Min Sketch Data Structure

In [6]:
cm_sketch = CountMinSketch(p,w)

s = 1000 sample points <br>
f = locality parameter

In [7]:
seed_np = np.random.seed(1)
s = 1000.0                                                      #sample size
sqrt_s = np.sqrt(s)

f = np.random.uniform(1.0/sqrt_s,1.0 - (1.0/sqrt_s))            #locality parameter
print(f)

0.422269993919


random vector of d dimensions dran uniformly at random from (0,f)

In [8]:
d = waveform_data.shape[1]                                         #dimension of the dataset
random_vector = np.random.uniform(0,f,d)
print(random_vector)

[  3.04171419e-01   4.82970534e-05   1.27665974e-01   6.19706091e-02
   3.89918179e-02   7.86520983e-02   1.45919926e-01   1.67542999e-01
   2.27526139e-01   1.77013265e-01   2.89347634e-01   8.63340503e-02
   3.70802645e-01   1.15649588e-02   2.83118311e-01   1.76215296e-01
   2.35917950e-01   5.92811917e-02   8.36523146e-02   3.38130404e-01
   4.08867810e-01   1.32349626e-01]


__waveform_randomnly = data sample of r dimensions__  

In [12]:
np.random.seed(1)
base =  np.maximum(2, 1/f)
r = int(np.random.uniform(1.0+(0.5*math.log(s,base)),math.log(s,base)))
waveform_randomly = waveform_data.sample(n=r,axis = 1,random_state=12)
print(waveform_randomly.head())

     7     10  21    15    14    8 
0  1.56  5.44   2  2.39  4.22  4.33
1  4.77  3.19   1 -0.24  2.14  5.15
2 -0.35 -0.99   2  5.47  4.23  0.06
3  5.26  4.15   1  0.12  2.00  2.19
4 -0.10  2.40   2  3.49  4.25  3.50


__sample_data = 1000 sampled data points of r dimens each__

In [14]:
sample_data = waveform_randomly.sample(n = int(s),random_state=12)
print(sample_data.head())
print(sample_data.shape)

        7     10  21    15    14    8 
3288 -1.01  2.71   2  3.81  4.93  0.76
3366  3.21  3.72   1  1.23  1.69  6.49
2037  2.43  2.74   1  1.97 -1.12  3.74
2881  4.67  2.55   1  1.08 -0.71  4.83
2999  1.73  4.08   2  3.68  3.77  0.16
(1000, 6)


__max and min valus from each of r dimensions__

In [15]:
col_max = np.max(sample_data,axis = 0)
col_min = np.min(sample_data,axis = 0)

__Normalize the sample data__

In [17]:
normalized_waveform_sample = (sample_data - col_min)/(col_max-col_min)
print(normalized_waveform_sample.head())

            7         10   21        15        14        8 
3288  0.120507  0.350365  1.0  0.664286  0.658800  0.257796
3366  0.566596  0.473236  0.5  0.401020  0.340216  0.853430
2037  0.484144  0.354015  0.5  0.476531  0.063913  0.567568
2881  0.720930  0.330900  0.5  0.385714  0.104228  0.680873
2999  0.410148  0.517032  1.0  0.651020  0.544739  0.195426


__find the points with all the features cooresponding to the points included in sample_data__

In [19]:
indexes_sample = sample_data.index
waveform_sample_rows = waveform_data.ix[indexes_sample]
print(indexes_sample)
print(waveform_sample_rows.head())

Int64Index([3288, 3366, 2037, 2881, 2999, 3300, 3221, 2870,  767,  483,
            ...
            3481,  853, 1280, 3338, 1403, 1698,  733,  574, 2032,  885],
           dtype='int64', length=1000)
        0     1     2     3     4     5     6     7     8     9  ...    12  \
3288 -0.06 -0.16 -0.72  0.08 -0.04  2.06  0.87 -1.01  0.76  1.76 ...  4.91   
3366  0.18  1.25  2.09  0.72  1.22  2.69  3.77  3.21  6.49  3.71 ...  3.62   
2037  0.53 -0.51  0.46  2.64  1.88  2.81  1.55  2.43  3.74  4.34 ...  3.27   
2881  0.29  0.82  2.47  2.23  4.88  5.53  6.19  4.67  4.83  4.34 ...  3.06   
2999 -1.24  0.64 -0.58  0.04 -0.40 -0.11  2.24  1.73  0.16  1.46 ...  4.30   

        13    14    15    16    17    18    19    20  21  
3288  4.74  4.93  3.81  4.34  3.22  2.22 -0.85  0.55   2  
3366  0.35  1.69  1.23  0.22  0.16 -1.07  0.73 -1.02   1  
2037  1.37 -1.12  1.97 -0.04 -0.85 -0.14  0.25  1.31   1  
2881  0.99 -0.71  1.08 -0.79 -0.44 -0.82  2.42  0.71   1  
2999  3.10  3.77  3.68  2.44  2.35  

__set dimension to -1 if that dimension is not included in sample_data__

In [20]:
included_cols = waveform_randomly.columns
Y_cap = waveform_sample_rows

for col in Y_cap:
    if col not in included_cols:
        Y_cap[col] = -1
    else:
        Y_cap[col] = np.floor((Y_cap[col] + random_vector[col] )/ f)
print(Y_cap.head())

      0   1   2   3   4   5   6   7   8   9  ...  12  13  14  15  16  17  18  \
3288  -1  -1  -1  -1  -1  -1  -1  -2   2  -1 ...  -1  -1  12   9  -1  -1  -1   
3366  -1  -1  -1  -1  -1  -1  -1   7  15  -1 ...  -1  -1   4   3  -1  -1  -1   
2037  -1  -1  -1  -1  -1  -1  -1   6   9  -1 ...  -1  -1  -2   5  -1  -1  -1   
2881  -1  -1  -1  -1  -1  -1  -1  11  11  -1 ...  -1  -1  -2   2  -1  -1  -1   
2999  -1  -1  -1  -1  -1  -1  -1   4   0  -1 ...  -1  -1   9   9  -1  -1  -1   

      19  20  21  
3288  -1  -1   5  
3366  -1  -1   2  
2037  -1  -1   2  
2881  -1  -1   2  
2999  -1  -1   5  

[5 rows x 22 columns]


Convert each point to a tuple and apply hash functions to it

In [21]:
data_tuples = [tuple(x) for x in Y_cap.to_records(index=False)]
for i in data_tuples:
    cm_sketch.add(i)

__ find min and max values from all dimens from points included in sample and Normalize the whole data __


In [22]:
col_max_whole = np.max(waveform_sample_rows,axis = 0)
col_min_whole = np.min(waveform_sample_rows,axis = 0)

Y_whole = waveform_data
norm_whole_data = (Y_whole - col_min_whole) / (col_max_whole - col_min_whole)

__set dimension to -1 if that dimension is not included in sample_data__

In [24]:
for col in Y_whole:
    if col not in included_cols:
        Y_whole[col] = -1
    else:
        Y_whole[col] = np.floor(Y_whole[col] + random_vector[col] / f)
print(Y_whole.head())

   0   1   2   3   4   5   6   7   8   9  ...  12  13  14  15  16  17  18  19  \
0  -1  -1  -1  -1  -1  -1  -1   1   4  -1 ...  -1  -1   4   2  -1  -1  -1  -1   
1  -1  -1  -1  -1  -1  -1  -1   5   5  -1 ...  -1  -1   2   0  -1  -1  -1  -1   
2  -1  -1  -1  -1  -1  -1  -1   0   0  -1 ...  -1  -1   4   5  -1  -1  -1  -1   
3  -1  -1  -1  -1  -1  -1  -1   5   2  -1 ...  -1  -1   2   0  -1  -1  -1  -1   
4  -1  -1  -1  -1  -1  -1  -1   0   4  -1 ...  -1  -1   4   3  -1  -1  -1  -1   

   20  21  
0  -1   2  
1  -1   1  
2  -1   2  
3  -1   1  
4  -1   2  

[5 rows x 22 columns]


In [25]:
print(Y_whole.shape)

(3509, 22)


convert each data point to tuple and apply hash functions

In [168]:
whole_data_tuples = [tuple(x) for x in Y_whole.to_records(index=False)]

for i in whole_data_tuples:
    cm_sketch.add(i)

In [142]:
print cm_sketch.tables

[array('l', [0, 0, 0, 0, 1, 0, 2, 1, 2, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 2, 1, 0, 0, 0, 3, 5, 0, 1, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 1, 0, 0, 3, 1, 0, 2, 0, 3, 0, 4, 4, 0, 1, 1, 1, 1, 0, 2, 2, 0, 0, 1, 0, 2, 0, 1, 3, 0, 1, 0, 0, 0, 0, 0, 2, 3, 0, 1, 3, 0, 0, 1, 2, 0, 2, 1, 3, 0, 0, 4, 1, 5, 1, 0, 1, 0, 1, 0, 0, 0, 2, 0, 0, 3, 0, 1, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 1, 3, 5, 2, 0, 1, 0, 3, 0, 2, 0, 0, 2, 0, 2, 0, 2, 0, 3, 0, 4, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 3, 4, 1, 0, 2, 0, 0, 0, 0, 0, 3, 1, 3, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 1, 0, 0, 1, 0, 0, 3, 4, 0, 0, 1, 0, 2, 0, 0, 3, 0, 1, 1, 0, 2, 0, 2, 0, 0, 1, 1, 0, 1, 5, 2, 0, 2, 1, 3, 0, 0, 0, 4, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 1, 2, 0, 0, 3, 2, 0, 2, 0, 0, 1, 0, 4, 0, 0, 1, 2, 2, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 